In [2]:
!pip install numpy pandas matplotlib seaborn nltk wordcloud tensorflow scikit-learn keras


  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 KB 25.9 MB/s eta 0:00:00
  Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 4.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.0 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached kiwisolver-1.4.7-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached pillow-11.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (4.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
# Importing necessary libraries for EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')

# Importing libraries necessary for Model Building and Training
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import warnings
warnings.filterwarnings('ignore')


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [4]:
data = pd.read_csv('/kaggle/input/spam-ds/spam.csv')
data.head()


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/spam-ds/spam.csv'

In [5]:
data['Spam']=data['Category'].apply(lambda x:1 if x=='spam' else 0)
data.head(5)

NameError: name 'data' is not defined

In [6]:
data.shape


NameError: name 'data' is not defined

In [7]:
sns.countplot(x='Spam', data=data)
plt.show()


NameError: name 'data' is not defined

In [8]:
# Downsampling to balance the dataset
ham_msg = data[data.Spam == 0]
spam_msg = data[data.Spam == 1]
ham_msg = ham_msg.sample(n=len(spam_msg),
						random_state=42)

# Plotting the counts of down sampled dataset
# Use pd.concat instead of append
balanced_data = pd.concat([ham_msg, spam_msg]).reset_index(drop=True)
plt.figure(figsize=(8, 6))
sns.countplot(data = balanced_data, x='Spam') # Changed 'spam' to 'Spam' to match column name
plt.title('Distribution of Ham and Spam email messages after downsampling')
plt.xlabel('Message types')

NameError: name 'data' is not defined

In [9]:
balanced_data['Message'] = balanced_data['Message'].str.replace('Subject', '')
balanced_data.head()


NameError: name 'balanced_data' is not defined

In [10]:
punctuations_list = string.punctuation
def remove_punctuations(text):
	temp = str.maketrans('', '', punctuations_list)
	return text.translate(temp)

balanced_data['Message']= balanced_data['Message'].apply(lambda x: remove_punctuations(x))
balanced_data.head()


NameError: name 'balanced_data' is not defined

In [11]:
def remove_stopwords(text):
	stop_words = stopwords.words('english')

	imp_words = []

	# Storing the important words
	for word in str(text).split():
		word = word.lower()

		if word not in stop_words:
			imp_words.append(word)

	output = " ".join(imp_words)

	return output


balanced_data['Message'] = balanced_data['Message'].apply(lambda text: remove_stopwords(text))
balanced_data.head()


NameError: name 'balanced_data' is not defined

In [12]:
def plot_word_cloud(data, typ):
	email_corpus = " ".join(data['Message'])

	plt.figure(figsize=(7, 7))

	wc = WordCloud(background_color='black',
				max_words=100,
				width=800,
				height=400,
				collocations=False).generate(email_corpus)

	plt.imshow(wc, interpolation='bilinear')
	plt.title(f'WordCloud for {typ} emails', fontsize=15)
	plt.axis('off')
	plt.show()

plot_word_cloud(balanced_data[balanced_data['Spam'] == 0], typ='Non-Spam')
plot_word_cloud(balanced_data[balanced_data['Spam'] == 1], typ='Spam')


NameError: name 'balanced_data' is not defined

In [13]:
data.head()

NameError: name 'data' is not defined

In [14]:
#train test split
train_X, test_X, train_Y, test_Y = train_test_split(balanced_data['Message'],
													balanced_data['Spam'],
													test_size = 0.2,
													random_state = 42)


NameError: name 'train_test_split' is not defined

In [15]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_X)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_X)
test_sequences = tokenizer.texts_to_sequences(test_X)

# Pad sequences to have the same length
max_len = 100 # maximum sequence length
train_sequences = pad_sequences(train_sequences,
								maxlen=max_len,
								padding='post',
								truncating='post')
test_sequences = pad_sequences(test_sequences,
							maxlen=max_len,
							padding='post',
							truncating='post')


NameError: name 'Tokenizer' is not defined

In [16]:
# Model 1 - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer # Import TfidfVectorizer

# Create a TfidfVectorizer to convert text to numerical features
vectorizer = TfidfVectorizer()
train_X_vec = vectorizer.fit_transform(train_X) # Fit and transform training data


# Create and train the RandomForestClassifier using the vectorized data
model1 = RandomForestClassifier()
model1.fit(train_X_vec,train_Y)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [17]:
test_X_vec = vectorizer.transform(test_X)


NameError: name 'vectorizer' is not defined

In [18]:
pred1 = model1.predict(test_X_vec)

NameError: name 'model1' is not defined

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [18]:
print("Random Forest Classifier")
print("Confusion Matrix: ")
print(confusion_matrix(test_Y, pred1)) # Change Y_test to test_Y
print("Accuracy: ", accuracy_score(test_Y, pred1))
print("--------------------------------")

Random Forest Classifier
Confusion Matrix: 
[[140   5]
 [ 17 137]]
Accuracy:  0.9264214046822743
--------------------------------


In [19]:
with open("random_forest_model.pkl", "wb") as file:
    pickle.dump(model1, file)

NameError: name 'pickle' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
text = "Hello Prajwal, you win a lottery of 1L"

# Transform the text using the same vectorizer used during training
X_new = vectorizer.transform([text])

# Make prediction
prediction = model1.predict(X_new)

# Output prediction
if prediction[0] == 1:
    print(True)
